In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import sympy as sp
from sympy.physics.vector import init_vprinting
from sympy import Symbol, symbols, Matrix, sin, cos, init_session, Derivative, diff, atan, tan, sqrt
init_vprinting(use_latex='mathjax', pretty_print=False)

vs, psis, dpsis, theta, dtheta, omega, domega, dts, xs, ys, zs, ws, lats, lons = symbols(r'v \psi \dot\psi theta \dot{\theta} omega \dot{\omega} T x y z w lat lon')
phi, dphi = symbols(r' phi \dot{\phi}')
theta1, theta2, l1, l2, theta, alpha, a, b, c, d = symbols('theta1 theta2 l1 l2 theta alpha a b c d')
alpha1, alpha2, alpha3, theta3, a1, a2, a3, a4, a5, r1, r2, r3, d1, d2, d3, pis, r, d = symbols('alpha1 alpha2 alpha3 theta3 a1 a2 a3 a4 a5 r1 r2 r3 d1 d2 d3 \pi r d')
dispx_s, dispy_s, dispz_s = symbols('dispx, dispy, dispz')
dotx, doty, dotz = symbols(r'\dot{x}, \dot{y}, \dot{z}')
deltaT = Symbol('\Delta t')
v = Matrix([[theta1, theta2, theta3, alpha1, alpha2, alpha3, a1, a2, a3, a4, a5, r1, r2, r3, d1, d2, d3, l1, l2, theta, alpha, a, d, phi, dphi, vs, psis, dpsis, theta, dtheta, omega, domega,dts, xs, ys, zs, dotx, doty, dotz, ws, lats, lons]])
display(v)

Matrix([[theta1, theta2, theta3, alpha1, alpha2, alpha3, a1, a2, a3, a4, a5, r1, r2, r3, d1, d2, d3, l1, l2, theta, alpha, a, d, phi, \dot{\phi}, v, \psi, \dot\psi, theta, \dot{\theta}, omega, \dot{\omega}, T, x, y, z, \dot{x}, \dot{y}, \dot{z}, w, lat, lon]])

In [52]:
dt = 0.5 # Time Step between Filter Steps
# model cinetique deltaT => dt
X = Matrix([[0,0,0,0,0,0]]).T
X_example = Matrix([[1,1,1,1,1,1]]).T
print("X\n")
display(X)
Z = Matrix([[xs,ys,dotx,doty]]).T

print("Z\n")
display(Z)
A = Matrix([[1.0, 0.0, deltaT, 0.0, sp.Rational(1,2)*deltaT**2, 0.0],
              [0.0, 1.0, 0.0, deltaT, 0.0, sp.Rational(1,2)*deltaT**2],
              [0.0, 0.0, 1.0, 0.0, deltaT, 0.0],
              [0.0, 0.0, 0.0, 1.0, 0.0, deltaT],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

print("A\n")
display(A)
# hessian
H = Matrix([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
               [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

print("H\n")
display(H)
covariance = 10**2
# matrice definissant la covariance , cov => covariance
cov = symbols('cov') # (10.0**2)
R = Matrix([[cov, 0.0, 0.0, 0.0],
            [0.0, cov, 0.0, 0.0],
            [0.0, 0.0, cov, 0.0],
            [0.0, 0.0, 0.0, cov]])

print("R\n")
display(R)
# matrice d'incertitude
inc = symbols("inc") # inc => 10
P = Matrix([[inc, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, inc, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, inc, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, inc, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, inc, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, inc]])

print("P\n")
display(P)
sa = 1.0
G = Matrix([[sp.Rational(1,2)*deltaT**2],
               [sp.Rational(1,2)*deltaT**2],
               [deltaT],
               [deltaT],
               [1.0],
               [1.0]])
Q = G*G.T*sa**2
Qs = Matrix([[0.5*deltaT**2],[0.5*deltaT**2],[deltaT],[deltaT],[1.0],[1.0]])

print("Q\n")
display(Q)
I = sp.Identity(6)



# étape de prediction
# on projette les données de départ (position, acceleration) dans la matrice qui contient le modéle cinétique
X = A*X_example

print("Prediction de X dans le modéle A: (A*X)")
display(X)
# on utilise A et P pr predire l'incertitude du modele tout en ajoutant Q qui est l'incertitude qui doit ce rajouter à chaque prédiction
P = A*P*A.T + Q

print("Calcul de la nouvelle incertitude P: (A*P*A.T) + Q")
display(P)


# étape de mise à jours
# calcul du gain de kalman

print("dans le calcul du gain de kalman S revient à calculer l'incertitude de la prediction + l'incertitude du capteur (S = H*P*H.T + R)")
S = H*P*H.T + R
display(S)

print("K c'est le gain de kalman, et K est donne la probabilité que la prediction ce trompe ou pas")
# K = (P@H.T)@S.pinv()
# display(K)


# Update the estimate via z
print("Y c'est le calcule de l'erreur entre la prediction et le capteur")
Y = Z - (H*X)                            # Innovation or Residual
# X = X + (K*Y)
display(Y)
print("X c'est la correction de la prediction par rapport au capteur, si le gain de kalman dit que la prediction ce trompe bcp, X seras bcp plus proche de ce que donne le capteur que de la prediction")
# display(X)
# Update the error covariance
# P = (I - (K*H))*P
# display(P)




X



Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

Z



Matrix([
[      x],
[      y],
[\dot{x}],
[\dot{y}]])

A



...

dans le calcul du gain de kalman S revient à calculer l'incertitude de la prediction + l'incertitude du capteur (S = H*P*H.T + R)


Matrix([
[0.25*\Delta t**4*inc + 0.25*\Delta t**4 + 1.0*\Delta t**2*inc + cov + 1.0*inc,                                                              0.25*\Delta t**4, 0.5*\Delta t**2*inc + 0.5*\Delta t**2,                       0.5*\Delta t**2],
[                                                             0.25*\Delta t**4, 0.25*\Delta t**4*inc + 0.25*\Delta t**4 + 1.0*\Delta t**2*inc + cov + 1.0*inc,                       0.5*\Delta t**2, 0.5*\Delta t**2*inc + 0.5*\Delta t**2],
[                                        0.5*\Delta t**2*inc + 0.5*\Delta t**2,                                                               0.5*\Delta t**2,                   cov + 1.0*inc + 1.0,                                   1.0],
[                                                              0.5*\Delta t**2,                                         0.5*\Delta t**2*inc + 0.5*\Delta t**2,                                   1.0,                   cov + 1.0*inc + 1.0]])

K c'est le gain de kalman, et K est donne la probabilité que la prediction ce trompe ou pas
Y c'est le calcule de l'erreur entre la prediction et le capteur


Matrix([
[-0.5*\Delta t**2 - 1.0*\Delta t + x - 1.0],
[-0.5*\Delta t**2 - 1.0*\Delta t + y - 1.0],
[                            \dot{x} - 1.0],
[                            \dot{y} - 1.0]])

X c'est la correction de la prediction par rapport au capteur, si le gain de kalman dit que la prediction ce trompe bcp, X seras bcp plus proche de ce que donne le capteur que de la prediction


$$Q = \begin{bmatrix}\sigma_{x}^2 & \sigma_{xy} & \sigma_{x \dot x} & \sigma_{x \dot y} & \sigma_{x \ddot x} & \sigma_{x \ddot y} \\ \sigma_{yx} & \sigma_{y}^2 & \sigma_{y \dot x} & \sigma_{y \dot y} & \sigma_{y \ddot x} & \sigma_{y \ddot y} \\ \sigma_{\dot x x} & \sigma_{\dot x y} & \sigma_{\dot x}^2 & \sigma_{\dot x \dot y} & \sigma_{\dot x \ddot x} & \sigma_{\dot x \ddot y} \\ \sigma_{\dot y x} & \sigma_{\dot y y} & \sigma_{\dot y \dot x} & \sigma_{\dot y}^2 & \sigma_{\dot y \ddot x} & \sigma_{\dot y \ddot y} \\ \sigma_{\ddot x x} & \sigma_{\ddot x y} & \sigma_{\ddot x \dot x} & \sigma_{\ddot x \dot y} & \sigma_{\ddot x}^2 & \sigma_{\ddot x \ddot y} \\ \sigma_{\ddot y x} & \sigma_{\ddot y y} & \sigma_{\ddot y \dot x} & \sigma_{\ddot y \dot y} & \sigma_{\ddot y \ddot x} & \sigma_{\ddot y}^2\end{bmatrix}$$

 $$Q = G\cdot G^T \cdot \sigma_v^2$$

In [31]:
from struct import unpack


class KF(object):
    def __init__(self, dt=0.5, sigmaModel=10, sigmaCapteur=10):
        self.dt = dt
        self.A = np.array([[1.0, 0.0, dt, 0.0, 1/2*dt**2, 0.0],
              [0.0, 1.0, 0.0, dt, 0.0, 1/2*dt**2],
              [0.0, 0.0, 1.0, 0.0, dt, 0.0],
              [0.0, 0.0, 0.0, 1.0, 0.0, dt],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])
        self.X = np.zeros((6,1))
        self.Z = np.zeros((4,1))
        self.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])
        self.cov = sigmaCapteur
        self.R = np.array([[self.cov**2, 0.0, 0.0, 0.0],
                           [0.0, self.cov **2, 0.0, 0.0],
                           [0.0, 0.0, self.cov**2, 0.0],
                           [0.0, 0.0, 0.0, self.cov**2]])
        self.inc = sigmaModel
        self.P = np.array([[self.inc, 0.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, self.inc, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.0, self.inc, 0.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, self.inc, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, self.inc, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, self.inc]])
        self.Q = np.array([[0.5*dt**2],[0.5*dt**2],[dt],[dt],[1.0],[1.0]])
        self.I = np.identity(6)

    def prediction(self):
        self.x = self.A*self.X
        self.P = self.A*self.P*self.A.T + self.Q    

    def update(self, measurement):
        self.Z[0] = measurement[0]
        self.Z[1] = measurement[1]
        self.Z[2] = measurement[2]
        self.Z[3] = measurement[3]
        self.S = self.H*self.P*self.H.T + self.R
        self.K = (self.P@self.H.T)@np.linalg.inv(self.S)
        self.Y = self.Z - (self.H@self.X)
        self.X = self.X + (self.K@self.Y)
        self.P = (self.I-(self.K@self.H))@self.P
        return self.Z
    
    def reset(self):
        self.A = np.array([[1.0, 0.0, self.dt, 0.0, 1/2*self.dt**2, 0.0],
              [0.0, 1.0, 0.0, self.dt, 0.0, 1/2*self.dt**2],
              [0.0, 0.0, 1.0, 0.0, self.dt, 0.0],
              [0.0, 0.0, 0.0, 1.0, 0.0, self.dt],
              [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])
        self.X = np.zeros((6.1))
        self.Z = np.zeros((4,1))
        self.H = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])
        self.R = np.array([[self.cov, 0.0, 0.0, 0.0],
                           [0.0, self.cov, 0.0, 0.0],
                           [0.0, 0.0, self.cov, 0.0],
                           [0.0, 0.0, 0.0, self.cov]])
        self.P = np.array([[self.inc, 0.0, 0.0, 0.0, 0.0, 0.0],
                           [0.0, self.inc, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.0, self.inc, 0.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, self.inc, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, self.inc, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, self.inc]])
        self.Q = np.array([[0.5*self.dt**2],[0.5*self.dt**2],[self.dt],[self.dt],[1.0],[1.0]])
        self.I = np.identity(6)

kf = KF()
date, \
time, \
millis, \
ax, \
ay, \
az, \
rollrate, \
pitchrate, \
yawrate, \
roll, \
pitch, \
yaw, \
speed, \
course, \
latitude, \
longitude, \
altitude, \
pdop, \
hdop, \
vdop, \
epe, \
fix, \
satellites_view, \
satellites_used, \
temp = np.genfromtxt("2014-02-14-002-Data.csv", delimiter=",", skip_header=True, unpack=True)

In [47]:
dlat = np.hstack((0.0, np.diff(latitude)))
dlon = np.hstack((0.0, np.diff(longitude)))
dt_s = np.hstack((0.0, np.diff(millis/1000.0)))

dy = 111.32 * np.cos(latitude * np.pi/180.0) * dlon # in km
dx = 111.32 * dlat # in km

mx = np.cumsum(1000.0 * dx) # in m
my = np.cumsum(1000.0 * dy) # in m
measurements = np.vstack((mx,my))

In [51]:
for i in range(measurements.shape[1]):
    print(measurements[:,i].reshape(2,1))

[[0.]
 [0.]]
[[0.]
 [0.]]
[[0.]
 [0.]]
[[0.]
 [0.]]
[[0.]
 [0.]]
[[-0.89056   ]
 [ 1.19018813]]
[[-0.89056   ]
 [ 1.19018813]]
[[-0.89056   ]
 [ 1.19018813]]
[[-0.89056   ]
 [ 1.19018813]]
[[-0.89056   ]
 [ 1.19018813]]
[[-1.78112   ]
 [ 2.38037646]]
[[-1.78112   ]
 [ 2.38037646]]
[[-1.78112   ]
 [ 2.38037646]]
[[-1.78112   ]
 [ 2.38037646]]
[[-1.78112   ]
 [ 2.38037646]]
[[-2.67168 ]
 [ 3.570565]]
[[-2.67168 ]
 [ 3.570565]]
[[-2.67168 ]
 [ 3.570565]]
[[-2.67168 ]
 [ 3.570565]]
[[-2.67168 ]
 [ 3.570565]]
[[-3.56224   ]
 [ 4.76075374]]
[[-3.56224   ]
 [ 4.76075374]]
[[-3.56224   ]
 [ 4.76075374]]
[[-3.56224   ]
 [ 4.76075374]]
[[-3.56224   ]
 [ 4.76075374]]
[[-4.4528    ]
 [ 5.95094269]]
[[-4.4528    ]
 [ 5.95094269]]
[[-4.4528    ]
 [ 5.95094269]]
[[-4.4528    ]
 [ 5.95094269]]
[[-4.4528    ]
 [ 5.95094269]]
[[-5.34336   ]
 [ 7.14113184]]
[[-5.34336   ]
 [ 7.14113184]]
[[-5.34336   ]
 [ 7.14113184]]
[[-5.34336   ]
 [ 7.14113184]]
[[-5.34336   ]
 [ 7.14113184]]
[[-6.23392  ]
 [ 8.331321